In [ ]:
!pip install -q pymongo
!pip install -q feature_engine

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import pymongo as pm
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xg
import pickle
import warnings
import json
warnings.filterwarnings('ignore')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.0/375.0 kB 5.1 MB/s eta 0:00:00


#### **Loading processed data**

In [ ]:
uri = "mongodb+srv://svbk:<pass>@cluster0.h5ef7.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0" #change password to access to the database
client = MongoClient(uri, server_api = ServerApi('1'))
db = client['VietNameseRealEstateData']
collection = db['Processed_Data']
final_data = list(collection.find())
df = pd.DataFrame(final_data)
df.drop(columns = '_id', inplace = True)
df

,Diện tích,Đường trước nhà,Mặt tiền,Số tầng,Số toilet,Số phòng ngủ,Số tầng missing,Số toilet missing,Số phòng ngủ missing,Đường trước nhà missing,...,Thành phố_điện biên,Thành phố_đà lạt,Thành phố_đà nẵng,Thành phố_đắk lắk,Thành phố_đồng nai,Thành phố_đồng tháp,Loại_CC,Loại_N,Loại_Đ,Mức giá
0,-0.700516,-0.975116,-0.303306,0.631913,-0.018601,-0.177704,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.243430
1,-0.544003,-0.975116,-0.303306,0.631913,-0.018601,-0.177704,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.085756
2,-0.800555,-0.975116,-0.648080,0.631913,-0.018601,-0.177704,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.243430
3,-0.544003,0.473555,0.048868,0.112935,-0.514920,-0.682911,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.152998
4,-0.523287,-0.975116,-0.648080,0.631913,-0.018601,-0.177704,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.152998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13623,2.444476,-0.424398,0.470432,-0.406044,-1.011238,0.327504,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.317356
13624,2.203357,0.082309,1.876781,-0.406044,-0.514920,0.327504,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.424996
13625,2.103318,-0.664968,0.216571,-0.406044,-0.514920,-0.177704,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.637873
13626,2.347259,-0.227839,1.444800,-0.406044,-0.514920,0.327504,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.272223


#### **Split the data into training and testing**

In [ ]:
X, y = df.drop(columns=['Mức giá']).to_numpy(), df['Mức giá'].to_numpy()
train_X, test_X, train_y, test_y = train_test_split(X, y,
                      test_size = 0.1, random_state = 42)

print(X.shape)
print(y.shape)
print(train_X.shape)
print(train_y.shape)
print(test_X.shape)
print(test_y.shape)

(13628, 73)
(13628,)
(12265, 73)
(12265,)
(1363, 73)
(1363,)


In [ ]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

def perform_gridsearch(X, y, param_grids, model, cv=9, scorer=['r2', 'neg_mean_squared_error']):

    kf = KFold(n_splits=cv, shuffle=True, random_state=42)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grids, cv=kf, scoring=scorer, refit='r2', n_jobs=-1)
    grid_search.fit(X, y)

    best_params = grid_search.best_params_
    results = grid_search.cv_results_
    best_models = grid_search.best_estimator_

    best_scores = {}
    for score_name in scorer:
        if score_name == 'r2':
            best_scores['best_r2_score'] = results['mean_test_r2'][results['rank_test_r2'] == 1][0]
        elif score_name == 'neg_mean_squared_error':
            best_scores['best_mse_score'] = -results['mean_test_neg_mean_squared_error'][results['rank_test_neg_mean_squared_error'] == 1][0]

    return {
        'best_params': best_params,
        'scores': best_scores,
        'best_model': best_models
    }

#### **Perform gridsearch**


In [ ]:
model = Ridge()
range1 = [0.0001, 0.001, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
range2 = list(range(2, 1001))
range1.extend(range2)
param_grids = {
    'alpha': range1,
    'fit_intercept': [True, False],
    'solver': ['sparse_cg']
}

In [ ]:

result = perform_gridsearch(train_X, train_y, param_grids, model, cv = 9)
with open('model_gridsearch.pkl', 'wb') as f:
    pickle.dump(result, f)
with open('model_gridsearch.pkl', 'rb') as f:
    exp = pickle.load(f)
loaded_model = exp['best_model']
y_test_pred = loaded_model.predict(test_X)
print(f"R2-score on test set: {r2_score(test_y, y_test_pred)}")
print(f"MSE on test set: {mean_squared_error(test_y, y_test_pred)}")
print(f"Best parameters: {exp['best_params']}")
print(f"Best R2-score: {exp['scores']['best_r2_score']}")
print(f"Best MSE score: {exp['scores']['best_mse_score']}")

R2-score on test set: 0.46657801230352147
MSE on test set: 0.5415213021062255
Best parameters: {'alpha': 8, 'fit_intercept': True, 'solver': 'sparse_cg'}
Best R2-score: 0.4682041373330508
Best MSE score: 0.5299873796823541
